In [7]:
using ModelingToolkit, DomainSets, Sophon, CairoMakie, Lux
using Optimization, OptimizationOptimJL

@parameters x, t
@variables f₁(..), f₂(..), θ₁(..), θ₂(..), w₁(..), w₂(..)
Dₜ = Differential(t)
Dₜ² = Differential(t)^2
Dₓ = Differential(x)
Dₓ² = Differential(x)^2

eqs = [
    Dₓ(θ₁(x,t)) - Dₓ²(w₁(x,t)) + Dₜ²(w₁(x,t)) + w₁(x,t) - w₂(x,t) ~ f₁(x,t),
    Dₓ(θ₂(x,t)) - Dₓ²(w₂(x,t)) + Dₜ²(w₂(x,t)) + w₂(x,t) - w₁(x,t) ~ f₂(x,t),
    Dₜ²(θ₁(x,t)) ~ Dₓ²(θ₁(x,t)) + Dₓ(w₁(x,t)) - θ₁(x,t),
    Dₜ²(θ₂(x,t)) ~ Dₓ²(θ₂(x,t)) + Dₓ(w₂(x,t)) - θ₂(x,t),
]

bcs = [
    θ₁(x, 0.0) ~ π/2*cos(x) + x - π/2,
    Dₜ(θ₁(x, 0.0)) ~ 0.0,
    w₁(x, 0.0) ~ π/2*sin(x),
    Dₜ(w₁(x, 0.0)) ~ 0.0,
    θ₂(x, 0.0) ~ cos(x) + 2/π * x - 1,
    Dₜ(θ₂(x, 0.0)) ~ 0.0,
    w₂(x, 0.0) ~ sin(x),
    Dₜ(w₂(x, 0.0)) ~ 0.0,
    θ₁(0.0, t) ~ 0.0,
    θ₁(π, t) ~ 0.0,
    w₁(0.0, t) ~ 0.0,
    w₁(π, t) ~ 0.0,
    θ₂(0.0, t) ~ 0.0,
    θ₂(π, t) ~ 0.0,
    w₂(0.0, t) ~ 0.0,
    w₂(π, t) ~ 0.0]

    
domains = [x ∈ Interval(0.0, π),
           t ∈ Interval(0.0, 1.0)]

@named timo_double = PDESystem(eqs, bcs, domains, [x,t], [f₁,f₂,θ₁,θ₂,w₁,w₂])

PDESystem
Equations: Equation[Differential(t)(Differential(t)(w₁(x, t))) + Differential(x)(θ₁(x, t)) + w₁(x, t) - Differential(x)(Differential(x)(w₁(x, t))) - w₂(x, t) ~ f₁(x, t), Differential(t)(Differential(t)(w₂(x, t))) + Differential(x)(θ₂(x, t)) + w₂(x, t) - Differential(x)(Differential(x)(w₂(x, t))) - w₁(x, t) ~ f₂(x, t), Differential(t)(Differential(t)(θ₁(x, t))) ~ Differential(x)(Differential(x)(θ₁(x, t))) + Differential(x)(w₁(x, t)) - θ₁(x, t), Differential(t)(Differential(t)(θ₂(x, t))) ~ Differential(x)(Differential(x)(θ₂(x, t))) + Differential(x)(w₂(x, t)) - θ₂(x, t)]
Boundary Conditions: Equation[θ₁(x, 0.0) ~ x + 1.5707963267948966cos(x) - 1.5707963267948966, Differential(t)(θ₁(x, 0.0)) ~ 0.0, w₁(x, 0.0) ~ 1.5707963267948966sin(x), Differential(t)(w₁(x, 0.0)) ~ 0.0, θ₂(x, 0.0) ~ 0.6366197723675814x + cos(x) - 1, Differential(t)(θ₂(x, 0.0)) ~ 0.0, w₂(x, 0.0) ~ sin(x), Differential(t)(w₂(x, 0.0)) ~ 0.0, θ₁(0.0, t) ~ 0.0, θ₁(π, t) ~ 0.0, w₁(0.0, t) ~ 0.0, w₁(π, t) ~ 0.0, θ₂(0.

In [8]:
pinn = PINN(θ₁ = FullyConnected(2,1, tanh; hidden_dims=16,num_layers=3),
            θ₂ = FullyConnected(2,1, tanh; hidden_dims=16,num_layers=3),
            w₁ = FullyConnected(2,1, tanh; hidden_dims=16,num_layers=3),
            w₂ = FullyConnected(2,1, tanh; hidden_dims=16,num_layers=3),
            f₁ = FullyConnected(2,1, tanh; hidden_dims=16,num_layers=3),
            f₂ = FullyConnected(2,1, tanh; hidden_dims=16,num_layers=3))

sampler = QuasiRandomSampler(2000, (ntuple(1000,8)..., ntuple(200,8)...))
strategy = NonAdaptiveTraining(1, 10)

prob = Sophon.discretize(timo_double, pinn, sampler, strategy)

OptimizationProblem. In-place: true
u0: ComponentVector{Float64}(θ₁ = (layer_1 = (weight = [0.31867825984954834 0.6138238310813904; 0.2613894045352936 0.6328346133232117; … ; 1.6346582174301147 1.376692771911621; -0.5471957921981812 -0.1772933453321457], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_2 = (weight = [0.22003386914730072 0.44067296385765076 … 0.1312142014503479 0.15264122188091278; 0.3249497711658478 0.25062045454978943 … -0.004338416270911694 -0.31324124336242676; … ; -0.19209200143814087 0.1941489428281784 … -0.5983707308769226 -0.4576032757759094; 0.2337038218975067 0.5827190279960632 … 0.5307864546775818 -0.3726338744163513], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_3 = (weight = [-0.6360896825790405 -0.2541704773902893 … 0.06904325634241104 -0.6883504986763; -0.23904269933700562 0.35655441880226135 … 0.499893456697464 0.0017520394176244736; … ; -0.46788400411605835 0.6969655156135559 … -0.6686041355133057 -0.0039729527197778225; 0.3680928647518158 -0.2859622538089752 … -

In [9]:
function callback(p, l)
    println("Loss: $l")
    return false
end

res = Optimization.solve(prob, BFGS(); maxiters=1000, callback=callback)

Loss: 698.972250951143


Loss: 115.02137786742772


Loss: 71.44264341015428


Loss: 48.73236880122541


Loss: 42.97533734916942


...

Loss: 1.0953140620758481e-5


Loss: 1.0915959931904027e-5


Loss: 1.0872379591597088e-5


Loss: 1.082292448586749e-5


Loss: 1.0760888598396818e-5


In [ ]:
f₁_analytic(x,t) = cos(t)*(1-sin(x))
f₂_analytic(x,t) = 2/π * cos(t)*(1-sin(x))
θ₁_analytic(x,t) = (π/2*cos(x) + x - π/2)*cos(t) 
θ₂_analytic(x,t) = (cos(x) + 2/π * x - 1)*cos(t)
w₁_analytic(x,t) = π/2*sin(x)*cos(t)
w₂_analytic(x,t) = sin(x)*cos(t)

In [ ]:
phi = pinn.phi
ps = res.u

xs, ts= [infimum(d.domain):0.01:supremum(d.domain) for d in timo.domain]

f₁ = [sum(phi.f₁(([x,t]), ps.f₁)) for x in xs, t in ts]
f₂ = [sum(phi.f₂(([x,t]), ps.f₂)) for x in xs, t in ts]
θ₁ = [sum(phi.θ₁(([x,t]), ps.θ₁)) for x in xs, t in ts]
θ₂ = [sum(phi.θ₂(([x,t]), ps.θ₂)) for x in xs, t in ts]
w₁ = [sum(phi.w₁(([x,t]), ps.w₁)) for x in xs, t in ts]
w₂ = [sum(phi.w₂(([x,t]), ps.w₂)) for x in xs, t in ts]

fig = Figure(resolution=(3000,500))
axis = (xlabel="t", ylabel="x", title="f1")

heatmap(fig[1, 1], xs, ts, f₁, axis=axis, colormap=:jet)
heatmap(fig[1, 2], xs, ts, f₂, axis= merge(axis, (; title="f2")), colormap=:jet)
heatmap(fig[2, 1], xs, ts, θ₁, axis=axis, colormap=:jet, merge(axis, (; title="θ1")))
heatmap(fig[2, 2], xs, ts, θ₂, axis= merge(axis, (; title="θ2")), colormap=:jet)
heatmap(fig[3, 1], xs, ts, w₁, axis=axis, colormap=:jet, merge(axis, (; title="w1")))
heatmap(fig[3, 2], xs, ts, w₂, axis= merge(axis, (; title="w2")), colormap=:jet)
display(fig)